In [5]:
import torch
#import torchvision
#import fastai
import os
import pandas as pd
import shutil
import random
import sklearn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
#import torchvision.transforms as transforms
from PIL import Image
import pydicom
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score
from skimage import morphology
from PIL import ImageOps
from sklearn.svm import SVC
import joblib
from sklearn.decomposition import PCA
from torch.utils.data import Dataset

In [2]:
csv_path = 'G:/MRI-challange/train.csv'
df = pd.read_csv(csv_path)
df.head()

,SeriesInstanceUID,prediction
0,1.3.46.670589.11.10042.5.0.6048.20240307143704...,0
1,1.3.46.670589.11.10042.5.0.6048.20240307143746...,0
2,1.3.46.670589.11.10042.5.0.6048.20240307143846...,0
3,1.3.46.670589.11.10042.5.0.4776.20240217030800...,0
4,1.3.46.670589.11.10042.5.0.4776.20240217030846...,0


In [7]:
os.mkdir('G:/MRI-challange/separated_data')


In [11]:
separated_data_folder = 'G:/MRI-challange/separated_data'
data_folder = 'G:/MRI-challange/data'

normal_folder = os.path.join(separated_data_folder, 'normal')
abnormal_folder = os.path.join(separated_data_folder, 'abnormal')

os.makedirs(normal_folder, exist_ok=True)
os.makedirs(abnormal_folder, exist_ok=True)

# Iterate through the CSV file and move folders
for index, row in df.iterrows():
    series_uid = row['SeriesInstanceUID']
    prediction = row['prediction']
    
    # Determine the source and destination paths
    src_folder = os.path.join(data_folder, series_uid)
    if prediction == 0:
        dst_folder = os.path.join(normal_folder, series_uid)
    else:
        dst_folder = os.path.join(abnormal_folder, series_uid)
    
    # Move the folder
    if os.path.exists(src_folder):
        shutil.copytree(src_folder, dst_folder)
    else:
        print(f"Folder {src_folder} does not exist")

print("Folders separated successfully!")

Folders separated successfully!


In [12]:
os.mkdir('G:/MRI-challange/splited_dataset')

In [13]:
# Paths to the normal and abnormal folders

base_folder = 'G:/MRI-challange/splited_dataset'

# Split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Create directories for train, val, and test sets
train_folder = os.path.join(base_folder, 'train')
val_folder = os.path.join(base_folder, 'val')
test_folder = os.path.join(base_folder, 'test')

for subset in ['train', 'val', 'test']:
    os.makedirs(os.path.join(train_folder, 'normal'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'abnormal'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'normal'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'abnormal'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'normal'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'abnormal'), exist_ok=True)

# Function to split and move folders
def split_and_move_folders(src_folder, dst_base_folder):
    folders = [f for f in os.listdir(src_folder) if os.path.isdir(os.path.join(src_folder, f))]
    random.shuffle(folders)
    
    train_count = int(len(folders) * train_ratio)
    val_count = int(len(folders) * val_ratio)
    test_count = len(folders) - train_count - val_count
    
    splits = {
        'train': folders[:train_count],
        'val': folders[train_count:train_count + val_count],
        'test': folders[train_count + val_count:]
    }
    
    for split, split_folders in splits.items():
        for folder in split_folders:
            src_path = os.path.join(src_folder, folder)
            dst_path = os.path.join(dst_base_folder, split, os.path.basename(src_folder), folder)
            shutil.move(src_path, dst_path)

# Split and move normal and abnormal folders
split_and_move_folders(normal_folder, base_folder)
split_and_move_folders(abnormal_folder, base_folder)

print("Data split into train, val, and test sets successfully!")

Data split into train, val, and test sets successfully!


In [ ]:
def process_and_save_dicom(src_folder, dst_folder):
    for root, _, files in os.walk(src_folder):
        for file in files:
            if file.endswith(".dcm"):
                dicom_path = os.path.join(root, file)
                dicom_data = pydicom.dcmread(dicom_path)
            if 'T2W_TSE'  in dicom_data.ProtocolName:

                # Extract metadata
                slice_orientation = dicom_data[(0x2001, 0x100b)].value
                if (slice_orientation != 'SAGITTAL') and (slice_orientation != 'CORONAL'):
                    pixel_array = dicom_data.pixel_array
                    protocol_name = dicom_data.ProtocolName.replace(" ", "_")  # Remove spaces in description
                    instance_number = dicom_data.InstanceNumber

                    # Extract Image Orientation (Patient) tag to determine left-right orientation
                    image_orientation = dicom_data[0x0020, 0x0037].value

                    # Check if the first element of image_orientation is negative
                    # This would indicate a left-to-right orientation, requiring a flip
                    if image_orientation[0] < 0:
                        # Flip the image horizontally to make it right-to-left
                        pixel_array = np.fliplr(pixel_array)

                    # Get the original folder name (parent of current root)
                    original_folder_name = os.path.basename(root)

                    # Create a new folder for the subject using original folder name + PatientID
                    subject_folder_name = f"{original_folder_name}"
                    subject_folder = os.path.join(dst_folder, subject_folder_name)
                    os.makedirs(subject_folder, exist_ok=True)

                    # Construct the save path with instance number in the filename
                    save_path = os.path.join(subject_folder, f"{os.path.splitext(file)[0]}_{protocol_name}_Instance{instance_number}.npy")

                    # Save the numpy array
                    np.save(save_path, pixel_array)


In [104]:
base_folder = 'G:/MRI-challange/splited_dataset'
output_folder = 'G:/MRI-challange/final_data_T2'

# Create output directories for train, val, and test sets
for subset in ['train', 'val', 'test']:
    for category in ['normal', 'abnormal']:
        os.makedirs(os.path.join(output_folder, subset, category), exist_ok=True)

# Process and save DICOM files in train, val, and test sets
for subset in ['train', 'val', 'test']:
    for category in ['normal', 'abnormal']:
        src_folder = os.path.join(base_folder, subset, category)
        dst_folder = os.path.join(output_folder, subset, category)
        process_and_save_dicom(src_folder, dst_folder)

print("DICOM files processed and saved as NumPy arrays with SeriesDescription in filenames.")

DICOM files processed and saved as NumPy arrays with SeriesDescription in filenames.


In [5]:
def extract_instance_number(filename):
    """Extract the instance number from the filename."""
    # Assuming the instance number is always preceded by 'Instance'
    base_name = os.path.splitext(os.path.basename(filename))[0]
    instance_str = base_name.split('_')[-1]  # Get the last part after splitting by '_'
    
    # Remove any non-digit characters from the instance part to extract the number
    instance_number = ''.join(filter(str.isdigit, instance_str))
    
    return int(instance_number)

def show_all_slices_of_random_subject(output_folder):
    # List all subject folders in the output directory
    subject_folders = [f.path for f in os.scandir(output_folder) if f.is_dir()]
    
    if not subject_folders:
        print("No subjects found in the output folder.")
        return
    
    # Select a random subject folder
    random_subject_folder = random.choice(subject_folders)
    print(f"Displaying all slices for subject: {os.path.basename(random_subject_folder)}")
    
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(random_subject_folder, f) for f in os.listdir(random_subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print("No .npy files found for the selected subject.")
        return
    
    # Sort files based on extracted instance numbers
    npy_files = sorted(npy_files, key=extract_instance_number)

    # Number of slices
    num_slices = len(npy_files)
    
    # Determine grid size (rows and columns) for displaying the slices
    num_columns = 5  # Display 5 images per row
    num_rows = (num_slices + num_columns - 1) // num_columns  # Calculate the number of rows needed
    
    # Create a figure with the calculated grid size
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 3))
    axes = axes.flatten()  # Flatten the axes array
    
    # Display each slice in the grid
    for i, file_path in enumerate(npy_files):
        # Load the image data from the .npy file
        image_data = np.load(file_path)
        
        # Extract the instance number using the defined function
        instance_number = extract_instance_number(file_path)
        
        # Display the image and set the title as the instance number
        axes[i].imshow(image_data, cmap='gray')
        axes[i].set_title(f"Instance {instance_number}")
        axes[i].axis('off')
    
    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

In [11]:
def crop_background(image, target_size=(256, 256)):
        """
        Crops the black background from the image using a tighter thresholding and bounding box method.
        Assumes that the image has dimensions (height, width, channels).
        """
        # If image has multiple channels, use only the first channel for thresholding
        if len(image.shape) == 3:  # (height, width, channels)
            image_2d = image[:, :, 0]  # Use the first channel for cropping
        else:
            image_2d = image

        # Adjusted thresholding: using a higher percentile for a tighter crop
        binary_image = image_2d > np.percentile(image_2d, 60)  # Increase percentile to 15 or higher if needed

        # Remove small connected components (noise) - optional
        binary_image = morphology.remove_small_objects(binary_image, min_size=100)

        # Visualize the binary image
        plt.imshow(binary_image, cmap='gray')
        plt.title("Tighter Binary Image")
        plt.show()

        # Find the bounding box
        coords = np.column_stack(np.where(binary_image))
        if coords.size == 0:
            # Handle the case where the image is completely black
            return image  # Return the original image if there's no valid crop

        min_row, min_col = coords.min(axis=0)
        max_row, max_col = coords.max(axis=0)

        # Apply a tighter crop using the bounding box
        cropped_image = image[min_row:max_row+1, min_col:max_col+1]

        # Convert cropped image to PIL Image and ensure it is in 'L' mode (grayscale)
        cropped_image = Image.fromarray(cropped_image.squeeze())
        cropped_image = ImageOps.grayscale(cropped_image)
        
        cropped_image = cropped_image.resize(target_size, Image.LANCZOS)
        cropped_image = np.array(cropped_image)

        # Ensure the resized image has the correct shape (target_size, target_size, channels)
        if len(cropped_image.shape) == 2:  # Ensure it's still 2D
            cropped_image = np.expand_dims(cropped_image, axis=-1)

        # Visualize the final tightly cropped and resized image
        plt.imshow(cropped_image.squeeze(), cmap='gray')
        plt.title(f"Final Cropped and Resized Image: {cropped_image.shape}")
        plt.show()

        return cropped_image


def process_subject_slices(subject_folder):
    """Process the slices of a subject, resize them, split into hemispheres, and return the array."""
    
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(subject_folder, f) for f in os.listdir(subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print("No .npy files found in the selected subject folder.")
        return None
    
    # Sort files based on extracted instance numbers
    npy_files = sorted(npy_files, key=extract_instance_number)
    
    # Limit to the first 16 slices, or pad with zeros if there are fewer
    num_slices = 16
    processed_slices = []
    
    for i in range(num_slices):
        if i < len(npy_files):
            image_data = np.load(npy_files[i])
        else:
            # Create a zero array if there are not enough slices
            image_data = np.zeros((288, 288))  # Assuming original slices are 288x288
            
            
        # CROPPING ADDED HERE
        image_data = crop_background(image_data)
        
        # Resize to 256x256
        resized_slice = cv2.resize(image_data, (256, 256), interpolation=cv2.INTER_AREA)
        
        processed_slices.append([resized_slice])
    
    # Convert to numpy array with shape (16, 2, 256, 128)
    processed_slices = np.array(processed_slices)
    
    return processed_slices

### new histogram func

In [29]:
def compute_histograms_for_subject(subject_folder, num_bins=255):
    # Get all .npy files in the subject's folder
    npy_files = [os.path.join(subject_folder, f) for f in os.listdir(subject_folder) if f.endswith(".npy")]
    
    if not npy_files:
        print(f"No .npy files found for the subject folder: {subject_folder}")
        return None
    
    # Sort files to maintain order (assuming filenames include instance numbers)
    npy_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1].replace("Instance", "")))

    all_histograms = []

    # We only want slices 5, 6, 7, and 8 (index 4 to 7)
    for i, file_path in enumerate(npy_files[4:8]):
        # Load the image data from the .npy file
        image_data = np.load(file_path)
        
        # Resize the image to 256x256
        resized_image = cv2.resize(image_data, (256, 256), interpolation=cv2.INTER_LINEAR)

        # Exclude zeros and compute histograms
        resized_image_nonzero = resized_image[resized_image > 0]

        resized_image_hist, _ = np.histogram(resized_image[resized_image > 0], bins=num_bins, range=(0, 255))
        
        # If no non-zero pixels are found, skip this slice
        if resized_image_hist.sum() == 0:
            print(f"Skipping slice with no non-zero pixels in file: {file_path}")
            continue
        
        # Normalize histogram to ensure equal contribution of each slice
        if resized_image_hist.sum() > 0:
            resized_image_hist = resized_image_hist / np.sum(resized_image_hist)
        
        all_histograms.append(resized_image_hist)
    
    # If no valid histograms were found, return None
    if not all_histograms:
        print(f"No valid histograms found for the subject folder: {subject_folder}")
        return None
    
    # Flatten the list of histograms into a single feature vector
    subject_histogram_vector = np.concatenate(all_histograms)
    
    return subject_histogram_vector


In [32]:
class BrainHistogramDataset(Dataset):
    def __init__(self, base_folder, num_bins=255):
        self.num_bins = num_bins
        
        # List all subject folders in both 'normal' and 'abnormal' directories
        normal_folder = os.path.join(base_folder, 'normal')
        abnormal_folder = os.path.join(base_folder, 'abnormal')
        
        self.subject_folders = []
        self.labels = []
        
        # Add normal subjects
        normal_subject_folders = [os.path.join(normal_folder, f) for f in os.listdir(normal_folder) if os.path.isdir(os.path.join(normal_folder, f))]
        self.subject_folders.extend(normal_subject_folders)
        self.labels.extend([0] * len(normal_subject_folders))  # Label 0 for normal
     
        
        # Add abnormal subjects
        abnormal_subject_folders = [os.path.join(abnormal_folder, f) for f in os.listdir(abnormal_folder) if os.path.isdir(os.path.join(abnormal_folder, f))]
        self.subject_folders.extend(abnormal_subject_folders)
        self.labels.extend([1] * len(abnormal_subject_folders))  # Label 1 for abnormal

    def __len__(self):
        return len(self.subject_folders)

    def __getitem__(self, idx):
        subject_folder = self.subject_folders[idx]
        histogram_vector = compute_histograms_for_subject(subject_folder, self.num_bins)
        
        # Handle cases where histogram_vector is None
        if histogram_vector is None:
            histogram_vector = np.zeros(16 * self.num_bins)
        
        label = self.labels[idx]
        return torch.tensor(histogram_vector, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
    
            
    def ret_label(self):
        return  self.labels

In [36]:
# Paths to your data folders
train_folder = 'G:/MRI-challange/final_data_T1/train'
val_folder = 'G:/MRI-challange/final_data_T1/val'

In [38]:
# Create datasets
train_dataset = BrainHistogramDataset(train_folder)
val_dataset = BrainHistogramDataset(val_folder)


In [40]:
def compute_sample_weights(train_dataset):
    # Extract labels from the dataset
    labels = [label for _, label in train_dataset]
    
    # Calculate the number of samples per class
    class_sample_counts = np.bincount(labels)
    
    # Compute class weights as the inverse of the class sample counts
    class_weights = 1.0 / (class_sample_counts + 1e-9)  # Adding a small value to avoid division by zero
    
    # Normalize class weights (optional)
    class_weights = class_weights / class_weights.sum() * len(class_weights)
    print(class_weights)
    
    # Create sample weights based on class weights
    sample_weights = np.array([class_weights[label] for label in labels])
    
    # Scale the weights if they are too small
    sample_weights *= len(sample_weights)  # Scale by the number of samples
    
    return sample_weights

In [42]:
# Example usage:
sample_weights = compute_sample_weights(train_dataset)

[0.23970944 1.76029056]


In [44]:
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)
print(device)

cuda


In [48]:
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [50]:
test_folder = 'G:/MRI-challange/final_data_T1/test'
test_dataset = BrainHistogramDataset(test_folder)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [52]:
train_labels = train_dataset.ret_label()
val_labels = val_dataset.ret_label()
test_labels = test_dataset.ret_label()

In [54]:

# Step to gather all histogram vectors first
def compute_all_histograms(dataset):
    all_histograms = []
    
    for idx in range(len(dataset)):
        subject_folder = dataset.subject_folders[idx]
        histogram_vector = compute_histograms_for_subject(subject_folder)
        
        if histogram_vector is not None:
            all_histograms.append(histogram_vector)
    
    return np.array(all_histograms)

# Step to fit and apply PCA after gathering all histograms
def apply_pca_to_histograms(all_histograms, target_dim=20):
    pca = PCA(n_components=target_dim)
    reduced_histograms = pca.fit_transform(all_histograms)
    return reduced_histograms, pca

# Use these functions after loading your dataset
all_histograms = compute_all_histograms(train_dataset)
reduced_histograms, r_pca = apply_pca_to_histograms(all_histograms)


In [56]:
def apply_pca_to_test_histograms(test_dataset, pca):
    all_histograms_test = compute_all_histograms(test_dataset)
    reduced_histograms_test = pca.transform(all_histograms_test)
    return reduced_histograms_test

reduced_histograms_val = apply_pca_to_test_histograms(val_dataset, r_pca)
print("done")
reduced_histograms_test = apply_pca_to_test_histograms(test_dataset, r_pca)


done


In [59]:
reduced_histograms_train = reduced_histograms
print(reduced_histograms_train.shape)
print(reduced_histograms_val.shape)
print(reduced_histograms_test.shape)

(826, 20)
(102, 20)
(107, 20)


In [62]:
class ReducedHistogramDataset(Dataset):
    def __init__(self, histograms, labels):
        self.histograms = histograms
        self.labels = labels

    def __len__(self):
        return len(self.histograms)

    def __getitem__(self, idx):
        histogram = self.histograms[idx]
        label = self.labels[idx]
        return torch.tensor(histogram, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Create PyTorch datasets
train_dataset = ReducedHistogramDataset(reduced_histograms_train, train_labels)
val_dataset = ReducedHistogramDataset(reduced_histograms_val, val_labels)
test_dataset = ReducedHistogramDataset(reduced_histograms_test, test_labels)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


In [60]:
def save_pca_model(pca, filename='pca_model_flair.pkl'):
    joblib.dump(pca, filename)
    print(f"PCA model saved as {filename}")

# Save the PCA model
save_pca_model(r_pca, 'pca_model_flair.pkl')

PCA model saved as pca_model_flair.pkl


In [64]:
from sklearn.preprocessing import StandardScaler
def train_svm(train_loader, val_loader,test_loader, kernel='rbf', C=1.0):
    # Prepare the training data from the train_loader
    X_train, y_train = [], []
    for inputs, labels in train_loader:
        X_train.extend(inputs.cpu().numpy())
        y_train.extend(labels.cpu().numpy())
    
    # Prepare the validation data from the val_loader
    X_val, y_val = [], []
    for inputs, labels in val_loader:
        X_val.extend(inputs.cpu().numpy())
        y_val.extend(labels.cpu().numpy())
    
    # Convert both training and validation data to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    
    # Combine training and validation data
    X_combined = np.concatenate([X_train, X_val])
    y_combined = np.concatenate([y_train, y_val])
    

    # Initialize and train the SVM model on the combined data
    svm_model = SVC(kernel=kernel, C=C, probability=True)  # Use probability=True to enable ROC/AUC computation
    svm_model.fit(X_combined, y_combined)

    # Validation
    X_test, y_test = [], []
    for inputs, labels in test_loader:
        X_test.extend(inputs.cpu().numpy())
        y_test.extend(labels.cpu().numpy())

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # Standardize validation data using the same scaler
   # X_test = scaler.transform(X_test)

    # Make predictions and compute probabilities
    preds = svm_model.predict(X_test)
    probs = svm_model.predict_proba(X_test)[:, 1]

    # Compute performance metrics
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    auc_score = roc_auc_score(y_test, probs)
    auc_pr = average_precision_score(y_test, probs)

    print(f'Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, AUC: {auc_score:.4f}, AUC-PR: {auc_pr:.4f}')
    return svm_model


In [ ]:
SVM_MODEL = train_svm(train_loader, val_loader,test_loader, kernel='poly', C=0.9)